In [145]:
import pandas as pd
import netCDF4 as nc
import numpy as np
from math import floor
from geopy import distance
tqdm.pandas()


In [20]:
grid = pd.DataFrame(columns = ["lat", "long", "population"])

GET GRIDS

In [24]:
import sys
sys.path.append("../tools")
# from grids import pxtolat, pxtolong

In [197]:
def pxtolong(px, width):
    adjlong = (px / width) * 360 
    return adjlong - 180

def pxtolat(px, height):
    adjlat = (px/height) * 180
    return -(adjlat -90)

def closest_solar(lat, lon, arr):
    est_lat = round(lat * 0.05) / 0.05
    est_long = round(lon * 0.05) / 0.05
    
    if est_lat == round(est_lat, 1):
        est_lat -= 0.05

    if est_long == round(est_long, 1):
        est_long -= 0.05

    return arr[(est_lat, est_long)] if (est_lat, est_long) in arr.keys() else None

def closest_winds(lat, lon, arr):
    # 361 576
    # width = 8640
    # height = 4320
    # min_dist = 10 ** 7
    # min_val = None
    # for i in range(len(arr)):
    #     for j in range(len(arr[0])):
    #         dist = distance.distance((lat, lon), (arr[i][j][0], arr[i][j][1])).km
            
    #         if dist < min_dist:
    #             min_dist = dist
    #             min_val = arr[i][j][2]
    est_lat = round(lat * 2) / 2
    est_long = round(lon / 0.625) * 0.625

    if est_lat == 0.0:
        est_lat = 0.5

    if est_long == 180.0:
        est_long = -1.0 * est_long
    if est_long == 0.0:
        est_long = 0.625
    
    return arr[(est_lat, est_long)]


In [63]:
from tqdm import tqdm

d = {
    'lat': [],
    'long': [],
    'population': []
}


with open('./gpw_pop.asc', 'r') as f:
        for i in range(6):
            f.readline() # skip pre data
        width = 8640
        height = 4320
        for rn in tqdm(range(0, 4320)):
            k = f.readline().split(' ')
            for c in range(0, 8640):
                cell_val = float(k[c]) # read population in grid

                if cell_val <= 0: # skip pixel if no population
                    continue
                # print(cell_val)
                d['lat'].append(pxtolat(rn, height))
                d['long'].append(pxtolong(c, width))
                d['population'].append(cell_val)
        
        
        grid = pd.DataFrame(d)

100%|██████████| 4320/4320 [00:27<00:00, 159.58it/s]


POPULATION

In [65]:
tmp = grid.loc[grid['population'] > 500000]
tmp.head(10)

,lat,long,population
2742573,45.791667,126.666667,621797.9
2926015,43.916667,125.250000,559883.8
2932979,43.833333,87.583333,520489.2
3108488,41.833333,123.333333,545899.1
3108490,41.833333,123.416667,601812.9
3168988,41.083333,28.833333,629407.6
3168989,41.083333,28.875000,543774.2
3172486,41.041667,28.791667,646545.8
3172487,41.041667,28.833333,818725.8
3172488,41.041667,28.875000,506676.8


EARTHQUAKES

TSUNAMI

HURRICANES

WIND

In [94]:

ds = nc.Dataset('../../energy_strength/wind/avg_wind_speed.nc')
lat = ds['lat'][:]
lon = ds['lon'][:]
speed = ds['M2TMNXFLX_5_12_4_SPEED'][:]



In [ ]:
res = {}
for i in tqdm(range(len(lat))):
    for j in range(len(lon)):
        res[(lat[i],lon[j])] = speed[i][j]
res

In [165]:
grid['wind_speed'] = grid.progress_apply(lambda x: closest_winds(x['lat'], x['long'], res), axis=1)

100%|██████████| 7550816/7550816 [02:04<00:00, 60412.69it/s]


In [166]:
grid.head(10)

,lat,long,population,wind_speed
0,83.166667,-74.208333,0.000009,4.253750
1,83.166667,-74.166667,0.000011,4.253750
2,83.166667,-74.125000,0.000011,4.253750
3,83.166667,-74.083333,0.000011,4.253750
4,83.166667,-74.041667,0.000008,4.266758
5,83.166667,-74.000000,0.000001,4.266758
6,83.125000,-76.916667,0.000002,4.527839
7,83.125000,-76.875000,0.000001,4.527839
8,83.125000,-74.666667,0.000002,4.253750
9,83.125000,-74.625000,0.000025,4.253750


In [ ]:
print(closest_winds(-90.0, -180.0, res))
res
# 8.378205

SOLAR

In [173]:
ds = nc.Dataset('../../energy_strength/solar/avg_solar_energy.nc')
lat = ds['lat'][:]
lon = ds['lon'][:]
energy = ds['FLDAS_NOAH01_CP_GL_M_001_SWdown_f_tavg'][:]

In [182]:
res = {}
for i in tqdm(range(len(lat))):
    for j in range(len(lon)):
        if not math.isnan(energy[i][j]):
            res[(round(lat[i], 2), round(lon[j], 2))] = energy[i][j]


  0%|          | 0/1500 [00:00<?, ?it/s]C:\Users\JtheMage\AppData\Local\Temp/ipykernel_9868/1224450979.py:4: UserWarning: Warning: converting a masked element to nan.
  if not math.isnan(energy[i][j]):
100%|██████████| 1500/1500 [02:01<00:00, 12.38it/s]


In [183]:
res

{(-55.85, -67.65): 97.825645,
 (-55.85, -67.55): 99.60057,
 (-55.85, -67.35): 101.34623,
 (-55.85, -67.25): 102.19874,
 (-55.85, -67.15): 100.71314,
 (-55.75, -67.45): 98.60866,
 (-55.75, -67.35): 99.10616,
 (-55.75, -67.25): 102.50005,
 (-55.65, -68.05): 96.16191,
 (-55.65, -67.65): 102.52526,
 (-55.65, -67.55): 101.33107,
 (-55.65, -67.45): 101.24142,
 (-55.65, -67.35): 103.76158,
 (-55.55, -68.65): 98.25454,
 (-55.55, -68.55): 101.05223,
 (-55.55, -68.25): 99.65891,
 (-55.55, -68.15): 97.82869,
 (-55.55, -68.05): 99.93344,
 (-55.55, -67.65): 106.09182,
 (-55.45, -69.55): 85.54212,
 (-55.45, -69.45): 87.90538,
 (-55.45, -69.35): 84.338104,
 (-55.45, -69.25): 89.058846,
 (-55.45, -69.15): 98.50018,
 (-55.45, -68.85): 91.66295,
 (-55.45, -68.75): 90.67084,
 (-55.45, -68.65): 88.80512,
 (-55.45, -68.55): 94.37677,
 (-55.45, -68.45): 93.302925,
 (-55.45, -68.35): 97.998085,
 (-55.45, -68.25): 98.51063,
 (-55.45, -68.15): 101.456665,
 (-55.35, -69.95): 85.95961,
 (-55.35, -69.75): 81.7749

In [176]:
ds['FLDAS_NOAH01_CP_GL_M_001_SWdown_f_tavg'][0][0]

masked

In [200]:
grid['solar'] = grid.progress_apply(lambda x: closest_solar(x['lat'], x['long'], res), axis=1)

100%|██████████| 7550816/7550816 [02:49<00:00, 44679.10it/s]


In [205]:
grid.loc[grid['solar'] < 100]

,lat,long,population,wind_speed,solar
12640,81.958333,-90.625000,0.000011,6.153218,99.887215
12641,81.958333,-90.583333,0.000027,6.153218,99.887215
12642,81.958333,-90.541667,0.000011,6.153218,99.887215
13288,81.916667,-90.666667,0.000006,6.153218,99.887215
13289,81.916667,-90.625000,0.000007,6.153218,99.887215
...,...,...,...,...,...
1986553,52.875000,-169.041667,0.203065,9.310513,99.148712
1986554,52.875000,-169.000000,0.177392,9.310513,99.148712
1986555,52.875000,-168.958333,0.035516,9.310513,99.148712
1990908,52.833333,-169.125000,0.059100,9.321427,99.148712


In [ ]:
grid.to_csv('grid.csv', index=False)